In [1]:
#import the textfile 
textFile = sc.textFile("/FileStore/tables/dataSet10.txt")

In [2]:
#split the text into verseID and words 
out= textFile.filter(lambda x: x!= '').map(lambda x: (x.split(' ',1)[0],x.split(' ',1)[1]) )
out.take(10)

In [3]:
#take out the punctuations, lowercase it, then split the words by whitespace 
def clean_split(string): 
  punctuations = [',', '.', ':', ';', '!', '(', ')', '?']
  string = string.lower()
  for pun in punctuations: 
    string = string.replace(pun, '')
  return string.split(' ')

out_2 = out.mapValues(clean_split)
out_2.take(10)

In [4]:
#Then map key to each word using flatMapValues
pair = out_2.flatMapValues(lambda x: x)
pair.take(20)


In [5]:
#Reverse the key-value in key-value pair to group by words 
reverse = pair.map(lambda x: (x[1], x[0]))
reverse.take(30)

In [6]:
#Get distinct words and accrue all the verseID associated with the word 
pair_distinct = reverse.groupByKey().filter(lambda x: x[0]!= '').mapValues(lambda x: list(set(x)))
pair_distinct.take(10)


In [7]:
#Sort by lexicographic order 
sorted_pair_distinct = pair_distinct.sortBy(lambda x: x[0])
sorted_pair_distinct.take(10)

In [8]:
#Save output File to S3 
sorted_pair_distinct.coalesce(1).saveAsTextFile("/mnt/%s/Output/Spark_HW2/Try2" %MOUNT_NAME )

In [9]:
#Extra credit - lines are ordered by the number of references in the index, and where this number is the same, order by the word being indexed.

order = sorted_pair_distinct.sortBy(lambda x: len(x[1]), ascending=False)
order.take(20)

In [10]:
#Save output File to S3 
order.coalesce(1).saveAsTextFile("/mnt/%s/Output/Spark_HW2/Try3" %MOUNT_NAME )